In [29]:
import os 
import shutil
import pickle
import datetime
import numpy as np
import pandas as pd
from datetime import date, timedelta

import matplotlib.pyplot as plt
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Load Data

### Mapping Index

In [30]:
path = '../data/processed'
infile = open(os.path.join(path,'index_to_cusId.pkl'),'rb')
index_to_id_dict = pickle.load(infile)
infile.close()

### Transaction

In [31]:
path = '../data/processed'
trans = pd.read_pickle(os.path.join(path,'transactions.pkl'))
trans["customer_id"] = trans["customer_id"].map(index_to_id_dict)
trans.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.0508,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.0305,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.0152,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.0169,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.0169,2


In [32]:
# customers
path = '../data/processed'
customers = pd.read_pickle(os.path.join(path,'customers.pkl'))
customers["customer_id"] = customers["customer_id"].map(index_to_id_dict)
customers.head(2)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0000,0.0000,ACTIVE,NONE,49.0000,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0000,0.0000,ACTIVE,NONE,25.0000,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...


## Top L1M

In [33]:
# start_dt =  datetime.datetime(2020,9,15)
end_dt =  datetime.datetime(2020,9,15)
start_dt = end_dt - timedelta(30)



temp_tran = trans[(trans.t_dat > start_dt) & (trans.t_dat <= end_dt)]
print('Min date: ', temp_tran.t_dat.min())
print('Max date: ', temp_tran.t_dat.max())
print(f'Total Customers: {temp_tran.customer_id.nunique()}')

top_l1m = pd.DataFrame(temp_tran.groupby(['article_id']).agg({'customer_id':'nunique'})\
                        .rename(columns={'customer_id':'total_customers'})\
                        .sort_values(by=['total_customers'],ascending=False)\
                        .reset_index()
                      )
top_l1m_list = list(top_l1m.iloc[:12]['article_id'])
top_l1m_list = ' '.join(top_l1m_list)
final_df = customers[["customer_id"]]
final_df['prediction'] = top_l1m_list

final_df.head()

Min date:  2020-08-17 00:00:00
Max date:  2020-09-15 00:00:00
Total Customers: 246099


<ipython-input-33-a12033eec785>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['prediction'] = top_l1m_list


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0751471001 0918292001 0915526001 0706016001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0751471001 0918292001 0915526001 0706016001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0751471001 0918292001 0915526001 0706016001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0918292001 0915526001 0706016001 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0751471001 0918292001 0915526001 0706016001 09...


In [34]:
final_df.to_csv(os.path.join(path,'top_l1m.csv'),index=False)

## Top L6Y

In [35]:
# start_dt =  datetime.datetime(2020,9,15)
end_dt =  datetime.datetime(2020,9,15)
start_dt = end_dt - timedelta(180)



temp_tran = trans[(trans.t_dat > start_dt) & (trans.t_dat <= end_dt)]
print('Min date: ', temp_tran.t_dat.min())
print('Max date: ', temp_tran.t_dat.max())
print(f'Total Customers: {temp_tran.customer_id.nunique()}')

top_l6m = pd.DataFrame(temp_tran.groupby(['article_id']).agg({'customer_id':'nunique'})\
                        .rename(columns={'customer_id':'total_customers'})\
                        .sort_values(by=['total_customers'],ascending=False)\
                        .reset_index()
                      )
top_l6m_list = list(top_l6m.iloc[:12]['article_id'])
top_l6m_list = ' '.join(top_l6m_list)
final_df = customers[["customer_id"]]
final_df['prediction'] = top_l6m_list

final_df.head()

Min date:  2020-03-20 00:00:00
Max date:  2020-09-15 00:00:00
Total Customers: 738206


<ipython-input-35-4904e2c9dc73>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['prediction'] = top_l6m_list


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0610776002 0706016001 0759871002 0372860002 06...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0610776002 0706016001 0759871002 0372860002 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0610776002 0706016001 0759871002 0372860002 06...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0610776002 0706016001 0759871002 0372860002 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0610776002 0706016001 0759871002 0372860002 06...


In [36]:
final_df.to_csv(os.path.join(path,'top_l6m.csv'),index=False)